In [10]:
import pandas as pd

# Mostrar todo el contenido de cada celda (sin recortes)
pd.set_option("display.max_colwidth", None)

We check to see if we can find the robots.txt file to determine whether web scraping is allowed.


revisamos a ver si encontramos el archivo robots.txt para saber si esta permitido realizar el web scrapy

In [11]:
import urllib.robotparser

url = "https://rentabien.com.co/inmuebles/"

rp = urllib.robotparser.RobotFileParser()
rp.set_url(url + "/robots.txt")
rp.read()

user_agent = "*"
pagina = url + "/ruta-que-quieres-scrapear"

if rp.can_fetch(user_agent, pagina):
    print("Está permitido hacer scraping en esta ruta")
else:
    print("NO está permitido hacer scraping en esta ruta")


Está permitido hacer scraping en esta ruta


Now we look for the file where the terms and conditions are located.



ahora buscamos el archivo donde esta termino y condicciones 

In [12]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://example.com"

posibles_rutas = [
    "/terms",
    "/terms-of-service",
    "/tos",
    "/legal",
    "/condiciones",
    "/terminos"
]

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; MiBot/1.0)"
}

def buscar_terminos():
    for ruta in posibles_rutas:
        url = BASE_URL + ruta
        try:
            response = requests.get(url, headers=headers, timeout=5)
            if response.status_code == 200:
                print(f"Página encontrada: {url}")
                return response.text
        except:
            pass
    print("No se encontró página de términos automáticamente")
    return None

def analizar_contenido(html):
    soup = BeautifulSoup(html, "html.parser")
    texto = soup.get_text().lower()

    palabras_clave = [
        "scraping",
        "data mining",
        "automat",
        "bot",
        "crawler",
        "robot"
    ]

    encontradas = [p for p in palabras_clave if p in texto]

    if encontradas:
        print("⚠️ Posibles restricciones encontradas:")
        for palabra in encontradas:
            print(" -", palabra)
    else:
        print("No se encontraron menciones directas a scraping")

html = buscar_terminos()

if html:
    analizar_contenido(html)


No se encontró página de términos automáticamente


We proceed with extracting the property links for their subsequent data extraction.


procedemos con la extraccion de los link de los inmuebles para su posterior extraccion de datos

In [13]:
 #etapa_1_guardar_enlaces.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Configurar navegador
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

url = "https://rentabien.com.co/inmuebles/"
driver.get(url)

WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "properties")))
time.sleep(3)

all_links = []
page_number = 1

while True:
    print(f"\n🟨 Página {page_number} cargada")

    # Esperar los inmuebles visibles
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='properties']//article")))
    except:
        print("No se encontraron inmuebles. Fin.")
        break

    # Extraer links
    inmuebles = driver.find_elements(By.XPATH, "//*[@id='properties']//article")
    for inm in inmuebles:
        try:
            link = inm.find_element(By.XPATH, ".//a[contains(@href, 'detalle')]").get_attribute("href")
            all_links.append(link)
        except:
            pass
    print(f"  ➕ Total acumulado: {len(all_links)} enlaces")

    # Intentar pasar a la siguiente página
    try:
        current = driver.find_element(By.CSS_SELECTOR, "a.page-numbers.current")
        next_page = str(int(current.text) + 1)

        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, f"//a[@class='page-numbers' and text()='{next_page}']"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        next_button.click()

        # Esperar que cambie contenido (nuevo primer inmueble)
        WebDriverWait(driver, 10).until(
            EC.staleness_of(inmuebles[0])
        )
        time.sleep(3)
        page_number += 1

    except:
        print("✅ No hay más páginas. Finalizando recolección de enlaces.")
        break

# Guardar los enlaces en un archivo CSV
with open("enlaces_inmuebles.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for link in all_links:
        writer.writerow([link])

print(f"\n✅ Total enlaces guardados: {len(all_links)} en 'enlaces_inmuebles.csv'")
driver.quit()



🟨 Página 1 cargada
  ➕ Total acumulado: 12 enlaces

🟨 Página 2 cargada
  ➕ Total acumulado: 24 enlaces

🟨 Página 3 cargada
  ➕ Total acumulado: 36 enlaces

🟨 Página 4 cargada
  ➕ Total acumulado: 48 enlaces

🟨 Página 5 cargada
  ➕ Total acumulado: 60 enlaces

🟨 Página 6 cargada
  ➕ Total acumulado: 72 enlaces

🟨 Página 7 cargada
  ➕ Total acumulado: 84 enlaces

🟨 Página 8 cargada
  ➕ Total acumulado: 96 enlaces

🟨 Página 9 cargada
  ➕ Total acumulado: 108 enlaces

🟨 Página 10 cargada
  ➕ Total acumulado: 120 enlaces

🟨 Página 11 cargada
  ➕ Total acumulado: 132 enlaces

🟨 Página 12 cargada
  ➕ Total acumulado: 144 enlaces

🟨 Página 13 cargada
  ➕ Total acumulado: 156 enlaces

🟨 Página 14 cargada
  ➕ Total acumulado: 168 enlaces

🟨 Página 15 cargada
  ➕ Total acumulado: 180 enlaces

🟨 Página 16 cargada
  ➕ Total acumulado: 192 enlaces

🟨 Página 17 cargada
  ➕ Total acumulado: 204 enlaces

🟨 Página 18 cargada
  ➕ Total acumulado: 216 enlaces

🟨 Página 19 cargada
  ➕ Total acumulado: 228


We proceed to extract the relevant data, but for privacy reasons we leave out the code and URL fields, since they are not useful for the analysis and excluding them is even good practice for the ML process. During the extraction, we also apply some data cleaning before saving the dataset.




procedemos a extraer los datos releveante pero por mayor privacidad dejamos fuera los datos de codigo y de URL el cual no es util en el analisis y para el proceso de ML es incluso una buena practica en la extraccion tambien aplicamos un poco de limpieza de datos antes de guardar

In [ ]:
# etapa_2_extraer_datos.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
from datetime import datetime
import re

# Leer enlaces
with open("enlaces_inmuebles.csv", "r", encoding="utf-8") as f:
    enlaces = [line.strip() for line in f if line.strip()]

#si queremos links parciales aqui los seleccionamos
#enlaces = enlaces[:555]
print(f"🔗 Total enlaces leídos: {len(enlaces)}")

# Configurar navegador en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

# Campos para el CSV (SIN Código ni URL)
campos = ["Tipo", "Operación", "Precio", "Administración", "Niveles", "Habitaciones", "Baños",
          "Parqueaderos", "Área Construida", "Área Lote", "Barrio", "Estrato", "Ciudad"]

def extraer_datos(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "domus_property"))
        )
    except TimeoutException:
        print(f"⚠️ Primer intento fallido, reintentando: {url}")
        driver.refresh()
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, "domus_property"))
            )
        except TimeoutException:
            print(f"❌ Falló definitivamente: {url}")
            return {k: "N/A" for k in campos}

    time.sleep(1)

    # Diccionario SIN URL
    datos = {k: "N/A" for k in campos}

    # Extraer tipo y operación
    try:
        titulo = driver.find_element(By.CSS_SELECTOR, "h3.entry-title").text.strip()
        partes = titulo.split(" en ")
        if len(partes) == 2:
            datos["Tipo"] = partes[0].strip()
            datos["Operación"] = partes[1].strip()
    except:
        pass

    # Precio
    try:
        datos["Precio"] = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[2]/div/div[2]/h5"
        ).text.strip()
    except:
        pass

    # Administración
    try:
        admin = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[2]/div/div[2]/p"
        ).text.strip()
        datos["Administración"] = admin.replace("Administración: ", "")
    except:
        pass

    # Características numéricas
    xpaths = {
        "Niveles": 1,
        "Habitaciones": 2,
        "Baños": 3,
        "Parqueaderos": 4,
        "Área Construida": 5,
        "Área Lote": 6,
    }

    for campo, i in xpaths.items():
        try:
            texto = driver.find_element(
                By.XPATH,
                f"//*[@id='domus_property']//div/div[3]/div/div[{i}]/div/div[2]"
            ).text.strip()
            datos[campo] = texto
        except:
            pass

    # Barrio (SIN Código)
    try:
        bloque = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div[@class='col-md-6']"
        ).get_attribute("innerHTML")
        barrio_match = re.search(r'Barrio:</b>\s*([^<]+)', bloque)
        if barrio_match:
            datos["Barrio"] = barrio_match.group(1).strip()
    except:
        pass

    # Estrato y Ciudad
    try:
        texto = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[5]/div[2]"
        ).text
        for linea in texto.split("\n"):
            if "Estrato:" in linea:
                datos["Estrato"] = linea.replace("Estrato:", "").strip()
            elif "Ciudad:" in linea:
                datos["Ciudad"] = linea.replace("Ciudad:", "").strip()
    except:
        pass

    return datos




def limpiar_numero(texto):
    if texto == "N/A":
        return texto

    texto = texto.strip()

    # Extraer número con posible decimal
    match = re.search(r"\d+[\.,]?\d*", texto)
    if not match:
        return "N/A"

    numero = match.group(0)

    # Si tiene coma como decimal, la convertimos a punto
    if "," in numero and "." not in numero:
        numero = numero.replace(",", ".")

    # Si tiene ambos (ej: 1.234,5) → punto miles, coma decimal
    elif "." in numero and "," in numero:
        numero = numero.replace(".", "").replace(",", ".")

    # Si solo tiene punto → lo dejamos tal cual (ej: 313.1)

    try:
        return str(float(numero))
    except:
        return "N/A"



def limpiar_valor_pesos(texto):
    if texto == "N/A":
        return texto
    return re.sub(r"[^\d]", "", texto)


datos_finales = []

for idx, enlace in enumerate(enlaces, 1):
    print(f"🔎 ({idx}/{len(enlaces)}) {enlace}")
    datos = extraer_datos(enlace)

    datos["Precio"] = limpiar_valor_pesos(datos["Precio"])
    datos["Administración"] = limpiar_valor_pesos(datos["Administración"])

    for campo in [
        "Niveles",
        "Habitaciones",
        "Baños",
        "Parqueaderos",
        "Área Construida",
        "Área Lote",
    ]:
        datos[campo] = limpiar_numero(datos[campo])

    datos_finales.append(datos)

# Guardar CSV final (SIN Código ni URL)
nombre_csv = f"inmuebles_detallado_{datetime.now().strftime('%Y%m%d')}.csv"
with open(nombre_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=campos)
    writer.writeheader()
    writer.writerows(datos_finales)

print(f"\n✅ Extracción completada. Guardado en: {nombre_csv}")
driver.quit()


🔗 Total enlaces leídos: 555
🔎 (1/555) https://rentabien.com.co/detalle?code=4256&alt=3524361
🔎 (2/555) https://rentabien.com.co/detalle?code=5140&alt=3524252
🔎 (3/555) https://rentabien.com.co/detalle?code=4255&alt=3524108
🔎 (4/555) https://rentabien.com.co/detalle?code=25327&alt=3523951
🔎 (5/555) https://rentabien.com.co/detalle?code=14980&alt=3523928
🔎 (6/555) https://rentabien.com.co/detalle?code=14979&alt=3523912
🔎 (7/555) https://rentabien.com.co/detalle?code=3231&alt=3523876
🔎 (8/555) https://rentabien.com.co/detalle?code=7594&alt=3523867
🔎 (9/555) https://rentabien.com.co/detalle?code=14978&alt=3523829
🔎 (10/555) https://rentabien.com.co/detalle?code=14977&alt=3523669
🔎 (11/555) https://rentabien.com.co/detalle?code=3646&alt=3523608
🔎 (12/555) https://rentabien.com.co/detalle?code=5139&alt=3523529
🔎 (13/555) https://rentabien.com.co/detalle?code=14976&alt=3523041
🔎 (14/555) https://rentabien.com.co/detalle?code=14975&alt=3523019
🔎 (15/555) https://rentabien.com.co/detalle?code=3

In [1]:
import pandas as pd
df = pd.read_csv('1.csv')
df.head(50)

,Tipo,Operación,Precio,Administración,Niveles,Habitaciones,Baños,Parqueaderos,Área Construida,Área Lote,Barrio,Estrato,Ciudad
0,Apartamento,Venta,5.350000e+08,680000.0,4.0,3.0,3.0,0.0,119.000,119.000,La Floresta De Suba,4.0,Bogotá
1,Oficina,Arriendo,1.500000e+06,362000.0,3.0,0.0,1.0,0.0,50.000,0.000,Chico Norte,6.0,Bogotá
2,Apartamento,Venta,5.300000e+08,280000.0,0.0,3.0,3.0,0.0,109.450,109.450,Marly,4.0,Bogotá
3,Apartamento,Venta,3.150000e+08,343000.0,2.0,1.0,1.0,1.0,30.000,0.000,Santa Barbara-Usaquén,5.0,Bogotá
4,Casa,Venta,2.300000e+08,NaN,1.0,3.0,2.0,1.0,90.000,106.000,Betania,3.0,Chinácota
5,Casa,Arriendo,6.500000e+05,NaN,0.0,2.0,1.0,0.0,106.000,106.000,BUENA VISTA III,2.0,Villa del rosario
6,Apartaestudio,Venta,4.260000e+08,1000000.0,0.0,1.0,1.0,1.0,38.570,38.570,Chico Reservado,6.0,Bogotá
7,Casa,Arriendo,2.700000e+06,NaN,0.0,4.0,3.0,2.0,126.000,126.000,Chía,4.0,Chía
8,Apartamento,Arriendo,9.000000e+05,NaN,0.0,2.0,2.0,0.0,65.000,65.000,ZULIMA I,3.0,Cúcuta
9,Local,Arriendo,1.300000e+07,NaN,0.0,0.0,2.0,0.0,150.000,90.000,Barrio Blanco,4.0,Cúcuta
